### Keras Tuner con el problema MNIST

Keras Tuner es una librería bastante reciente que simplifica en gran medida el ajuste de los hiperparámetros de una red neuronal. Toda la documentación en este enlace:

https://keras-team.github.io/keras-tuner/

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

Carga de los datos:

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from time import time
import shutil

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Normalización:

In [5]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Lo primero que hay que hacer es definir un hipermodelo, que es una función que genera un modelo de Keras que depende de unos hiperparámetros con los que vamos a jugar. Los hiperparámetros se muestrean a partir del argumento ``hp`` de la función.

En este ejemplo sólo vamos a ajustar la constante de regularización de la capa oculta:

In [6]:
def model_builder(hp):
  hp_lambda = hp.Choice('lambda', values = [1.0, 0.1, 0.01, 0.001, 0.0001])
  hp_lr = hp.Choice('lr', values = [1.0, 0.1, 0.01, 0.001, 0.0001])

  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = 50, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_lr),
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])

  return model

Borramos el directorio de logs:

In [7]:
!rm -rf my_dir/intro_to_kt/

Lo siguiente es crear un ``tuner`` para hacer el ajuste de los hiperparámetros. Existen distintos tipos:

- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn

Lo más fácil es hacer una búsqueda aleatoria con ``RandomSearch``. Al crear el ``tuner`` hay que especificar:

- El hipermodelo.
- La variable a optimizar.
- El número total de pruebas.
- El número de ejecuciones por prueba.

In [8]:
tuner = kt.RandomSearch(model_builder,
                        objective='val_acc',
                        max_trials=10,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

Un resumen del espacio de búsqueda:

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
lambda (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}
lr (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}


Y lanzamos la búsqueda:

In [10]:
tuner.search(img_train, label_train,
             epochs=1,
             validation_data=(img_test, label_test))

Trial 10 Complete [00h 00m 21s]
val_acc: 0.6632999976476034

Best val_acc So Far: 0.9364666740099589
Total elapsed time: 00h 03m 56s


Acceso al mejor modelo. Hay que tener en cuenta que ya está entrenado, y siempre es mejor reentrenarlo con todos los datos.

In [11]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(img_test, label_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.2936 - acc: 0.9384


[0.29360270500183105, 0.9383999705314636]

Resumen de los resultados:

In [12]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 05 summary
Hyperparameters:
lambda: 0.0001
lr: 0.01
Score: 0.9364666740099589

Trial 00 summary
Hyperparameters:
lambda: 0.01
lr: 0.001
Score: 0.9240000049273173

Trial 03 summary
Hyperparameters:
lambda: 0.001
lr: 0.0001
Score: 0.8992666800816854

Trial 08 summary
Hyperparameters:
lambda: 0.1
lr: 0.001
Score: 0.878433346748352

Trial 04 summary
Hyperparameters:
lambda: 1.0
lr: 0.001
Score: 0.8236666520436605

Trial 07 summary
Hyperparameters:
lambda: 0.1
lr: 0.01
Score: 0.8134666681289673

Trial 09 summary
Hyperparameters:
lambda: 1.0
lr: 0.01
Score: 0.6632999976476034

Trial 02 summary
Hyperparameters:
lambda: 0.01
lr: 0.1
Score: 0.5364666680494944

Trial 01 summary
Hyperparameters:
lambda: 0.001
lr: 1.0
Score: 0.09803333381811778

Trial 06 summary
Hyperparameters:
lambda: 0.01
lr: 1.0
Score: 0.09529999891916911


Obtenemos los parámetros del mejor modelo y lo reentrenamos:

In [13]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3190 - acc: 0.9215 - val_loss: 0.2724 - val_acc: 0.9443
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2893 - acc: 0.9412 - val_loss: 0.2831 - val_acc: 0.9463
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2829 - acc: 0.9452 - val_loss: 0.2866 - val_acc: 0.9446
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2779 - acc: 0.9484 - val_loss: 0.2830 - val_acc: 0.9472
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2761 - acc: 0.9483 - val_loss: 0.2946 - val_acc: 0.9464
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2756 - acc: 0.9487 - val_loss: 0.2955 - val_acc: 0.9410
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2732 - acc: 0.9491 - val_loss: 0.2523 - val_acc: 0.9541
Epoch 8/10
1875/1875 [==============================] - 5s 3ms

#### Otro ejemplo:

Ajuste del learning rate, el parámetro de regularización y el número de neuronas en la capa oculta con un tuner de tipo hyperband.

https://arxiv.org/pdf/1603.06560.pdf

Hipermodelo:

In [14]:
def model_builder_2(hp):
  hp_lambda = hp.Choice('lambda', values = [0.001, 0.0001])
  hp_units = hp.Int('units', min_value = 32, max_value = 128, step = 32)
  hp_learning_rate = hp.Choice('learning_rate', values = [1.0, 0.1, 0.01, 0.001])

  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])

  return model

Borramos la carpeta de logs:

In [15]:
!rm -rf my_dir/intro_hyperband/

Creamos el tuner:

In [16]:
tuner = kt.Hyperband(model_builder_2,
                     objective = 'val_acc',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_hyperband')

Resumen del espacio de búsqueda:

In [17]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
lambda (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 1.0, 'conditions': [], 'values': [1.0, 0.1, 0.01, 0.001], 'ordered': True}


Búsqueda:

In [18]:
tuner.search(img_train, label_train,
             epochs=10,
             validation_data=(img_test, label_test))

Trial 30 Complete [00h 01m 23s]
val_acc: 0.9746000170707703

Best val_acc So Far: 0.9775999784469604
Total elapsed time: 00h 15m 56s


Mejores hiperparámetros:

In [19]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lambda': 0.0001,
 'units': 96,
 'learning_rate': 0.001,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

Reentrenamiento del modelo:

In [20]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2947 - acc: 0.9206 - val_loss: 0.1771 - val_acc: 0.9544
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1520 - acc: 0.9630 - val_loss: 0.1381 - val_acc: 0.9671
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1226 - acc: 0.9719 - val_loss: 0.1249 - val_acc: 0.9705
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1063 - acc: 0.9773 - val_loss: 0.1261 - val_acc: 0.9702
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0977 - acc: 0.9798 - val_loss: 0.1160 - val_acc: 0.9744
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0904 - acc: 0.9819 - val_loss: 0.1256 - val_acc: 0.9714
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0856 - acc: 0.9840 - val_loss: 0.1168 - val_acc: 0.9749
Epoch 8/10
1875/1875 [==============================] - 7s 4m